In [2]:
import json
import warnings
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, date_format, max, count, collect_list, mean, stddev
from statsmodels.tsa.arima.model import ARIMA
import pandas as pd
from datetime import timedelta

In [4]:
spark = SparkSession.builder.appName("AsistenciaClaseForecast").getOrCreate()

25/01/11 15:28:43 WARN Utils: Your hostname, MacBook-Air-de-Alberto.local resolves to a loopback address: 127.0.0.1; using 192.168.0.13 instead (on interface en0)
25/01/11 15:28:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/11 15:29:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [52]:
csv_input = "asistencia_reducido.csv"
csv_nulos = "nulos.csv"
csv_filtrado = "filtrado.csv"
csv_input_mal = "asistencia_reducido_mal.csv"

In [53]:
# Procesar datos
def procesar_datos(spark, csv_asistencia, csv_nulos, csv_filtrado):
    # Lee el archivo CSV y transforma en df
    df = spark.read.csv(csv_asistencia, header=True, inferSchema=True)

    # Filtrar valores nulos y guardar en un archivo CSV
    df_nulos = df.filter(" OR ".join([f"{col} IS NULL" for col in df.columns]))
    df_nulos.write.csv(csv_nulos, header=True, mode="overwrite")

    # Filtrar valores válidos de estado y limpiar los datos
    valores_validos = ["presente", "tarde", "ausente"]
    df_filtrado = df.dropna().filter(col("estado_asistencia").isin(valores_validos))
    df_filtrado = df_filtrado.orderBy("timestamp", "asignatura", "estado_asistencia")
    df_filtrado.write.csv(csv_filtrado, header=True, mode="overwrite")

    # Mostrar el DataFrame filtrado
    print("Datos filtrados y ordenados:")
    df_filtrado.show()

    return df_filtrado

In [ ]:
registro_asistencia = procesar_datos(spark, csv_input, csv_nulos, csv_filtrado)

In [57]:
def test_calcular_metricas_y_forecast_json(df):
    # Obtener asignaturas una única vez
    asignaturas_df = df.select("asignatura").distinct()
    # Lista para almacenar resultados
    data_json = []

    # No hace falta revisar que las filas coincidan porque si no estuviesen falla en procesar_datos()

    asignaturas_df = df.select("asignatura").distinct()
    # (Mantenemos la misma lógica, .collect() porque no esperamos datos gigantes)
    asignaturas = [row["asignatura"] for row in asignaturas_df.collect()]

    # Comprobamos que están las asignaturas del csv reducido que hemos creado para ver las cosas más facil 
    for asig in asignaturas:
        if asig == "Matemáticas":
            print("OK - Se encontró 'Matemáticas'.")
        elif asig == "Historia":
            print("OK - Se encontró 'Historia'.")
        elif asig == "Biología":
            print("OK - Se encontró 'Biología'.")
        else:
            print(f"FAIL - Se encontró una asignatura inesperada: '{asig}'. Revisar el CSV.")

    

    for asignatura_row in asignaturas_df.collect():
        asignatura = asignatura_row["asignatura"]
        print(f"Asignatura: {asignatura}")

        # Filtramos la asignatura
        df_asignatura = df.filter(col("asignatura") == asignatura)
        
        # Último día
        ultimo_dia = df_asignatura.agg(
            max(date_format(col("timestamp"), "yyyy-MM-dd")).alias("ultimo_dia")
        ).first()["ultimo_dia"]
        print(f"Último día de registros: {ultimo_dia}")
        
        # Registros del último día
        df_ultimo_dia = df_asignatura.filter(
            date_format(col("timestamp"), "yyyy-MM-dd") == ultimo_dia
        )
        
        # Calculo de totales del último día
        totales_estado = df_ultimo_dia.groupBy("estado_asistencia").agg(
            count("alumno").alias("total"),
            collect_list("alumno").alias("alumnos")
        ).collect()

        # Diccionario de totales del día actual
        totales_dia_actual = {
            "presentes": next((row["total"] for row in totales_estado
                               if row["estado_asistencia"] == "presente"), 0),
            "tarde": next((row["total"] for row in totales_estado
                           if row["estado_asistencia"] == "tarde"), 0),
            "ausentes": next((row["total"] for row in totales_estado
                              if row["estado_asistencia"] == "ausente"), 0),
        }
        
        alumnos_por_estado = {
            "presentes": next((row["alumnos"] for row in totales_estado
                               if row["estado_asistencia"] == "presente"), []),
            "tarde": next((row["alumnos"] for row in totales_estado
                           if row["estado_asistencia"] == "tarde"), []),
            "ausentes": next((row["alumnos"] for row in totales_estado
                              if row["estado_asistencia"] == "ausente"), []),
        }
        
        totales_dia_actual["alumnos_por_estado"] = alumnos_por_estado
        
        # Mostramos en pantalla
        print("Totales del día actual:")
        print(totales_dia_actual)

        # Calcular estadísticas acumuladas
        conteos_diarios = df_asignatura.groupBy(
            "estado_asistencia",
            date_format(col("timestamp"), "yyyy-MM-dd").alias("fecha")
        ).agg(count("alumno").alias("total_diario"))
        
        estadisticas_acumuladas = conteos_diarios.groupBy("estado_asistencia").agg(
            mean("total_diario").alias("media"),
            stddev("total_diario").alias("desviacion")
        )
        
        print("Estadísticas acumuladas:")
        estadisticas_acumuladas.show()
        
        estadisticas_dict = estadisticas_acumuladas.collect()
        
        estadisticas_final = {
            "media_presentes": next((row["media"] for row in estadisticas_dict
                                     if row["estado_asistencia"] == "presente"), 0),
            "desviacion_presentes": next((row["desviacion"] for row in estadisticas_dict
                                          if row["estado_asistencia"] == "presente"), 0),
            "media_tarde": next((row["media"] for row in estadisticas_dict
                                 if row["estado_asistencia"] == "tarde"), 0),
            "desviacion_tarde": next((row["desviacion"] for row in estadisticas_dict
                                      if row["estado_asistencia"] == "tarde"), 0),
            "media_ausentes": next((row["media"] for row in estadisticas_dict
                                    if row["estado_asistencia"] == "ausente"), 0),
            "desviacion_ausentes": next((row["desviacion"] for row in estadisticas_dict
                                         if row["estado_asistencia"] == "ausente"), 0),
        }
        
        print("Diccionario de estadísticas calculadas:")
        print(estadisticas_final)

In [58]:
prueba = test_calcular_metricas_y_forecast_json(registro_asistencia)

OK - Se encontró 'Matemáticas'.
OK - Se encontró 'Historia'.
OK - Se encontró 'Biología'.
Asignatura: Matemáticas
Último día de registros: 2025-01-11
Totales del día actual:
{'presentes': 1, 'tarde': 2, 'ausentes': 1, 'alumnos_por_estado': {'presentes': ['Alumno9'], 'tarde': ['Alumno10', 'Alumno11'], 'ausentes': ['Alumno12']}}
Estadísticas acumuladas:
+-----------------+------------------+------------------+
|estado_asistencia|             media|        desviacion|
+-----------------+------------------+------------------+
|         presente|1.6666666666666667|0.5773502691896258|
|            tarde|1.3333333333333333|0.5773502691896258|
|          ausente|               1.0|               0.0|
+-----------------+------------------+------------------+

Diccionario de estadísticas calculadas:
{'media_presentes': 1.6666666666666667, 'desviacion_presentes': 0.5773502691896258, 'media_tarde': 1.3333333333333333, 'desviacion_tarde': 0.5773502691896258, 'media_ausentes': 1.0, 'desviacion_ausen

In [60]:
# Matemáticas
# presente
# 2 presentes , 2 presentes  , 1 presente || (2+2+1)/3 = 1.67
    # Distancias al cuadrado:
    # (2 - 1.6667)^2 = 0.1111
    # (2 - 1.6667)^2 = 0.1111
    # (1 - 1.6667)^2 = 0.4444
        # Suma = 0.6666
        # Varianza = 0.6666 / (3 - 1) = 0.3333
        # Desviación = (0.3333)*0.5 ≈ 0.57735

# tarde
# 1 presentes , 1 presentes  , 2 presente || (1+1+2)/3 = 1.33
    # Distancias al cuadrado:
    # (1 - 1.3333)^2 = 0.1111
    # (1 - 1.3333)^2 = 0.1111
    # (2 - 1.3333)^2 = 0.4444
        # Suma = 0.6666
        # Varianza = 0.6666 / (3 - 1) = 0.3333
        # Desviación = (0.3333)*05 ≈ 0.57735

# ausente
# 1 presentes , 1 presentes  , 1 presente || 1